In [13]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [14]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [15]:
%run -i ../../zkstats/core.py

In [16]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')

=======================  ZK-STATS FLOW =======================

In [17]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

data = np.array(json.loads(open(data_path, "r").read())['col_name'])
data_tensor = torch.reshape(torch.tensor(data),(1,-1, 1))

create_dummy(data_path, dummy_data_path)
dummy_data = np.array(json.loads(open(dummy_data_path, "r").read())['col_name'])
dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1,-1,1))

gt20_dummy_data_tensor = dummy_data_tensor[dummy_data_tensor > 20].reshape(1,-1,1)
dummy_theory_output = torch.exp(torch.mean(torch.log(gt20_dummy_data_tensor)))
gt20_data_tensor = data_tensor[data_tensor > 20].reshape(1,-1,1)
theory_output = torch.exp(torch.mean(torch.log(gt20_data_tensor)))


In [ ]:
scales = [8]
selected_columns = ['col_name']
commitment_maps = get_data_commitment_maps(data_path, scales)

In [18]:
# Verifier/ data consumer side: send desired calculation
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)

    def forward(self,X):
        # where part
        filter = (X>20)
        num_fil_X = torch.sum(filter.double())
        fil_X = torch.where(filter, X, 1.0)
        return (torch.abs((torch.log(self.w)*num_fil_X)-torch.sum(torch.log(fil_X)))<=num_fil_X*torch.log(torch.tensor(1.01)), self.w)

verifier_define_calculation(dummy_data_path, selected_columns, sel_dummy_data_path,verifier_model, verifier_model_path)

/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_11911/721953102.py:13: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  return (torch.abs((torch.log(self.w)*num_fil_X)-torch.sum(torch.log(fil_X)))<=num_fil_X*torch.log(torch.tensor(1.01)), self.w)


In [19]:
# prover calculates settings, send to verifier
print("Theory_output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = theory_output, requires_grad = False)

    def forward(self,X):
        # where part
        filter = (X>20)
        num_fil_X = torch.sum(filter.double())
        fil_X = torch.where(filter, X, 1.0)
        print('self w: ', self.w)
        return (torch.abs((torch.log(self.w)*num_fil_X)-torch.sum(torch.log(fil_X)))<=num_fil_X*torch.log(torch.tensor(1.01)), self.w)


prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model,prover_model_path, scales, "resources", settings_path)

Theory_output:  tensor(48.1276, dtype=torch.float64)
self w:  Parameter containing:
tensor(48.1276, dtype=torch.float64)
==== Generate & Calibrate Setting ====
scale:  [8]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":8,"param_scale":8,"scale_rebase_multiplier":10,"lookup_range":[-196442,45056],"logrows":18,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":3615,"total_const_size":306,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,8],"model_input_scales":[8],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"Div":{"denom":100.49927}},{"Ln":{"scale":256.0}},{"GreaterThan":{"a":0.0}}],"check_mode":"UNSAFE","version":"7.0.0","num_blinding_factors":null}


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_11911/1804498770.py:15: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  return (torch.abs((torch.log(self.w)*num_fil_X)-torch.sum(torch.log(fil_X)))<=num_fil_X*torch.log(torch.tensor(1.01)), self.w)


In [20]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2


Time setup: 29.234658002853394 seconds
Theory output:  tensor(48.1276, dtype=torch.float64)
!@# compiled_model exists? False
!@# compiled_model exists? True
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 48.12890625
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[11768814371718170976, 435173728250646979, 519717007263840094, 1741290966923863957], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [13008514816417923429, 2889748934031881905, 13995698908509180341, 3273700224560439773]]], 'proof': '24c94fb9e5feeb94860417ea624f01c352f8ea642a8d6e0a3c4d8d368db597c022e990f7d6bf962de6aa94825eabb3aca2c27b60ddcba9ba239bb844f216bd3109f896ad7bb99cb60bf25d4fe5d580508462a969927893945ff3b49b8a83f17d24ad89e221e392fc67416f4daa39cfc2dea35a5b6f07bfacc815ded5cbd2687628ac4a049c3ef6809e32663b463fefd1329dc6ca4005e4646f1a3afbffa5ac2514fe94e940e5d33543872483b33fbf273739ef03546509f658c0923acbd7a87814fb54b9e6f4ea3c24eb99b2c14a2cefc9ba5d415fdae39a7cc0c4dafe42f4683027d8055217c0fde3e4951f6d8398c4bba159217e3866764b2a994bab850c9d1db6f078672efcce6456d0ce5b26f1a4a51dc89f9eea0efa5f4fa4242bef444a206d7aa7358c2dbbe6691616707135106c29e5d1de40b5ee1dd748173c6390530db7ef323c9b80542fc69bb8c0bc2d9870fe62b8c8a5316b6777b175961fc16c

In [21]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path, selected_columns, commitment_maps)

num_inputs:  1
prf instances:  [[[11768814371718170976, 435173728250646979, 519717007263840094, 1741290966923863957], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [13008514816417923429, 2889748934031881905, 13995698908509180341, 3273700224560439773]]]
proof boolean:  1.0
proof result 1 : 48.12890625
verified
